# Анализ результатов

Загрузим два тестовых датасета - без шума и с возможным шумом.

In [1]:
import pandas as pd
import pickle
from sklearn.metrics import classification_report
import numpy as np

In [2]:
speakers = pd.read_csv('../data/dev-clean/LibriTTS/speakers.tsv',
                       skiprows=[0],
                       header=None,
                       sep='\t')
speakers.columns = ['id', 'gender', 'subset', 'name']

In [3]:
test_clean_path = '../data/test-clean/LibriTTS/test-clean'
test_other_path = '../data/test-other/LibriTTS/test-other'

### SVM

In [4]:
from src.data import *

In [5]:
test_clean_files = dataset_wav_paths(test_clean_path)
test_other_files = dataset_wav_paths(test_other_path)

In [6]:
label2idx = {'M': 0, 'F': 1}

# preparing test-clean
X_clean, y_clean, ids = data_features(test_clean_files, speakers, max_files_per_speaker=None,
                                                sr=24000, max_sig_duration=2)
X_clean = np.stack([x.flatten() for x in X_clean], axis=0)
y_clean = [label2idx[x] for x in y_clean]

# preparing test-other
X_other, y_other, _ = data_features(test_other_files, speakers, max_files_per_speaker=None,
                                                sr=24000, max_sig_duration=2)
X_other = np.stack([x.flatten() for x in X_other], axis=0)
y_other = [label2idx[x] for x in y_other]

100%|██████████| 33/33 [00:27<00:00,  1.21it/s]


In [7]:
svm_model = pickle.load(open('../models/svm.pkl', 'rb'))

Оценка качества на тестовом датасете без шума

In [8]:
clean_preds = svm_model.predict(X_clean)
print(classification_report(y_clean, clean_preds))

              precision    recall  f1-score   support

           0       0.97      0.89      0.93      1907
           1       0.93      0.98      0.96      2930

    accuracy                           0.95      4837
   macro avg       0.95      0.94      0.94      4837
weighted avg       0.95      0.95      0.95      4837



Оценка качества на тестовом датасете с возможным шумом

In [9]:
other_preds = svm_model.predict(X_other)
print(classification_report(y_other, other_preds))

              precision    recall  f1-score   support

           0       0.98      0.71      0.82      2561
           1       0.77      0.99      0.87      2559

    accuracy                           0.85      5120
   macro avg       0.88      0.85      0.85      5120
weighted avg       0.88      0.85      0.85      5120



Как и ожидалось, на данных, где появляется шум, модель справляется хуже - качество падает с 0.95 до 0.85.

Еще интересно посмотреть, какие записи плохо классифицировались.

In [10]:
test_clean_speakers = speakers[speakers['subset'] == 'test-clean'].copy()

# id говорящих, кол-во неправильно классифицированных записей (для датасета clean)
right_preds = np.unique(np.array(ids)[clean_preds == y_clean], return_counts=True)
# id говорящих, кол-во правильно классифицированных записей (для датасета clean)
wrong_preds = np.unique(np.array(ids)[clean_preds != y_clean], return_counts=True)

test_clean_speakers['right_preds'] = test_clean_speakers['id'].apply(lambda x: int(*right_preds[1][right_preds[0] == str(x)]))
test_clean_speakers['wrong_preds'] = test_clean_speakers['id'].apply(lambda x: int(*wrong_preds[1][wrong_preds[0] == str(x)]))

test_clean_speakers['predicted_label'] = test_clean_speakers.apply(lambda x : label2idx[x.gender] if x.right_preds > x.wrong_preds else {'M':1, 'F':0}[x.gender], axis=1)
test_clean_speakers['true_label'] = test_clean_speakers.apply(lambda x: int(label2idx[x.gender]), axis=1)

test_clean_speakers = test_clean_speakers[(test_clean_speakers['right_preds'] + test_clean_speakers['wrong_preds']) > 0]

In [11]:
test_clean_speakers

,id,gender,subset,name,right_preds,wrong_preds,predicted_label,true_label
33,61,M,test-clean,Paul-Gabriel Wiener,1,0,0,0
70,121,F,test-clean,Nikolle Doolin,87,18,1,1
125,237,F,test-clean,rachelellen,279,0,1,1
141,260,M,test-clean,Brad Bush,190,30,0,0
286,672,M,test-clean,Taylor Burton-Edward,1,8,1,0
358,908,M,test-clean,Sam Stinson,26,4,0,0
415,1089,M,test-clean,Peter Bobbe,173,17,0,0
444,1188,M,test-clean,Duncan Murrell,47,1,0,0
448,1221,F,test-clean,Dianne,7,0,1,1
474,1284,F,test-clean,Daniel Anaya,73,7,1,1


Можно так же посмотреть на метрики классификации по говорящим,  а не по их речам (считаю, что если больше речей одного говорящего получило правильную метку класса, то классификация правильная, если больше речей за другую метку класса - неправильная).

In [12]:
print(classification_report(test_clean_speakers['true_label'], test_clean_speakers['predicted_label']))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95        20
           1       0.90      1.00      0.95        19

    accuracy                           0.95        39
   macro avg       0.95      0.95      0.95        39
weighted avg       0.95      0.95      0.95        39



Получается, что на самом деле, для большинства людей, в целом, их пол был классифицирован верно (кроме двух мужчин).

Послушав некоторые плохо классифицированных говорящих, появились небольшие догадки, почему так произошло. Кажется, что, например, ошибки в классификации мужских голосов чаще происходят для говорящих, у которых голоса чуть повыше, чем у других (хотя для меня на слух они все равно 100% мужские).

### VGG-16

In [13]:
from src.train_cnn import AudioDataset, VGG

In [14]:
import torch

model = VGG('VGG16')
model.load_state_dict(torch.load('../models/vgg16.pkl'))

<All keys matched successfully>

In [15]:
test_clean_data = AudioDataset('../data/dev-clean/LibriTTS/speakers.tsv', test_clean_path)
test_other_data = AudioDataset('../data/dev-clean/LibriTTS/speakers.tsv', test_other_path)

100%|██████████| 33/33 [00:29<00:00,  1.13it/s]


In [16]:
model.eval()
device=torch.device('cuda')
model.to(device)

# creating dataloaders for  both test sets
clean_dataloader = torch.utils.data.DataLoader(test_clean_data, batch_size=32, shuffle=False)
other_dataloader = torch.utils.data.DataLoader(test_other_data, batch_size=32, shuffle=False)

clean_preds = []
other_preds = []

# model predictions
for X, y in clean_dataloader:
    X = X.to(device)
    probas = model(X).cpu().detach().numpy()
    clean_preds.extend(probas.flatten().tolist())

for X, y in other_dataloader:
    X = X.to(device)
    probas = model(X).cpu().detach().numpy()
    other_preds.extend(probas.flatten().tolist())

Результаты для тестовой выборки без шума

In [17]:
print(classification_report(test_clean_data.y, [round(x) for x in clean_preds]))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96      1907
           1       0.99      0.95      0.97      2930

    accuracy                           0.97      4837
   macro avg       0.96      0.97      0.96      4837
weighted avg       0.97      0.97      0.97      4837



Результаты для тестовой выборки с возможным шумом

In [18]:
print(classification_report(test_other_data.y, [round(x) for x in other_preds]))

              precision    recall  f1-score   support

           0       0.97      0.89      0.93      2561
           1       0.90      0.97      0.93      2559

    accuracy                           0.93      5120
   macro avg       0.93      0.93      0.93      5120
weighted avg       0.93      0.93      0.93      5120



VGG-16 немного лучше работает, чем SVM, на данных без шума (0.97 против 0.95). И значительно лучше на данных, где вероятно есть шум - 0.93 по сравнению с 0.85.

Опять-таки можно посмотреть на предсказания по говорящим. Опять сделаем это лишь для данных без шума

In [19]:
test_clean_speakers = speakers[speakers['subset'] == 'test-clean'].copy()
ids = test_clean_data.ids

# id говорящих, кол-во неправильно классифицированных записей (для датасета clean)
right_preds = np.unique(np.array(ids)[np.array([round(x) for x in clean_preds]) == test_clean_data.y], return_counts=True)
# id говорящих, кол-во правильно классифицированных записей (для датасета clean)
wrong_preds = np.unique(np.array(ids)[np.array([round(x) for x in clean_preds]) != test_clean_data.y], return_counts=True)

test_clean_speakers['right_preds'] = test_clean_speakers['id'].apply(lambda x: int(*right_preds[1][right_preds[0] == str(x)]))
test_clean_speakers['wrong_preds'] = test_clean_speakers['id'].apply(lambda x: int(*wrong_preds[1][wrong_preds[0] == str(x)]))

test_clean_speakers['predicted_label'] = test_clean_speakers.apply(lambda x : label2idx[x.gender] if x.right_preds > x.wrong_preds else {'M':1, 'F':0}[x.gender], axis=1)
test_clean_speakers['true_label'] = test_clean_speakers.apply(lambda x: int(label2idx[x.gender]), axis=1)

test_clean_speakers = test_clean_speakers[(test_clean_speakers['right_preds'] + test_clean_speakers['wrong_preds']) > 0]

In [20]:
test_clean_speakers

,id,gender,subset,name,right_preds,wrong_preds,predicted_label,true_label
33,61,M,test-clean,Paul-Gabriel Wiener,1,0,0,0
70,121,F,test-clean,Nikolle Doolin,93,12,1,1
125,237,F,test-clean,rachelellen,279,0,1,1
141,260,M,test-clean,Brad Bush,213,7,0,0
286,672,M,test-clean,Taylor Burton-Edward,7,2,0,0
358,908,M,test-clean,Sam Stinson,30,0,0,0
415,1089,M,test-clean,Peter Bobbe,188,2,0,0
444,1188,M,test-clean,Duncan Murrell,48,0,0,0
448,1221,F,test-clean,Dianne,7,0,1,1
474,1284,F,test-clean,Daniel Anaya,57,23,1,1


In [21]:
test_clean_speakers[test_clean_speakers['predicted_label'] != test_clean_speakers['true_label']]

,id,gender,subset,name,right_preds,wrong_preds,predicted_label,true_label


При такой оценке классификатора, результаты идеальные :) Для всех говорящих, VGG оценивает бОльшую часть речей правильной меткой пола.